## simpleなモデルでの実験
- SVM
- MLP(Multilayer perceptron)

In [2]:
import pickle
import warnings
import numpy as np 
import pandas as pd
from tqdm import tqdm
from sklearn import svm 
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import wandb 

import utils

warnings.simplefilter('ignore')
wandb.login()

def load_data(testfile, trait):
    train_data = []
    test_data = []
    path = '../data/Hazumi_features/Hazumi1911_features.pkl'
    TS, TS_ternary, TP, TP_binary, TP_cluster, text, audio, visual, vid = pickle.load(open(path, 'rb'), encoding='utf-8')
    for file in vid:
        data_t = pd.DataFrame(text[file])
        data_a = pd.DataFrame(audio[file])
        data_v = pd.DataFrame(visual[file])
        seq_len = len(TS_ternary[file])
        label = [TP_binary[file][trait] for _ in range(seq_len)]
        label = pd.DataFrame(label)
        data = pd.concat([data_t, data_a, data_v, label], axis=1)
        if file == testfile:
            test_data.append(data)
        else:
            train_data.append(data)
    train_data = pd.concat(train_data)
    test_data = pd.concat(test_data) 
    train_data = train_data.sample(frac=1, random_state=0)
    test_data = test_data.sample(frac=1, random_state=0)

    x_train, y_train, x_test, y_test =\
         train_data.iloc[:, :-1].values, train_data.iloc[:, -1].values, test_data.iloc[:, :-1].values, test_data.iloc[:, -1].values

    scaler = preprocessing.MinMaxScaler()
    # scaler = preprocessing.StandardScaler()

    x_train = scaler.fit_transform(x_train) 
    x_test = scaler.transform(x_test)

    return x_train, y_train, x_test, y_test

/home/ryoyanagimoto/anaconda3/envs/personality/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: r-yanagimoto. Use `wandb login --relogin` to force relogin


In [8]:
# SVM
files = utils.get_files()
traits = utils.get_traits()
iter = 5
project_name = 'svm' 
group_name = utils.randomname(5)
config = {
    "C": 100,
    "gamma": 0.001,
    "kernel": "sigmoid"
}

for _ in range(iter):
    for testfile in files:
        wandb.init(project=project_name, group=group_name, config=config, name=testfile)
        result = []
        for i in range(5):
            x_train, y_train, x_test, y_test = load_data(testfile, i)

            clf = svm.SVC(C=config["C"], gamma=config["gamma"], kernel=config["kernel"]) 
            history = clf.fit(x_train, y_train) 

            result.append(1 if clf.score(x_test, y_test) >= 0.5 else 0)

        print(f'{testfile} : {result}')
        wandb.log({
            'acc': sum(result) / len(result),
            '1extr': result[0],
            '2agre': result[1], 
            '3cons': result[2], 
            '4neur': result[3],
            '5open': result[4]
        })

1911F6002 : [0, 1, 0, 0, 0]


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0
2agre,1
3cons,0
4neur,0
5open,0


1911F6003 : [1, 1, 1, 1, 1]


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,1
2agre,1
3cons,1
4neur,1
5open,1


...

1911M5002 : [1, 0, 0, 1, 1]


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,1
2agre,0
3cons,0
4neur,1
5open,1


1911M6001 : [0, 1, 0, 1, 0]


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0
2agre,1
3cons,0
4neur,1
5open,0


In [ ]:
# MLP
files = utils.get_files()
traits = utils.get_traits()

for i in range(5):
    result = []
    for testfile in tqdm(files):
        x_train, y_train, x_test, y_test = load_data(testfile, i)

        clf = MLPClassifier(hidden_layer_sizes=(256, 256), solver='sgd', learning_rate='adaptive',\
             alpha=0.0001, activation='tanh', early_stopping=True) 
        history = clf.fit(x_train, y_train) 

        result.append(1 if clf.score(x_test, y_test) >= 0.5 else 0)

    print(f'{traits[i]} : {sum(result) / len(result)}')

In [ ]:
# MLP
files = utils.get_files()
traits = utils.get_traits()
iter = 5
project_name = 'mlp' 
group_name = utils.randomname(5)
config = {
    "hidden_layer_size": (256, 256),
    "solver": 'sgd',
    "learning_rate": 'adaptive',
    "alpha": 0.0001,
    "activation": 'tanh',
    "early_stopping": True
}

for _ in range(iter):
    for testfile in files:
        wandb.init(project=project_name, group=group_name, config=config, name=testfile)
        result = []
        for i in range(5):
            x_train, y_train, x_test, y_test = load_data(testfile, i)

            clf = MLPClassifier(hidden_layer_sizes=config["hidden_layer_size"], solver=config["solver"],
                                learning_rate=config["learning_rate"], alpha=config["alpha"],
                                activation=config["activation"], early_stopping=config["early_stopping"])
            history = clf.fit(x_train, y_train) 

            result.append(1 if clf.score(x_test, y_test) >= 0.5 else 0)

        print(f'{testfile} : {result}')
        wandb.log({
            'acc': sum(result) / len(result),
            '1extr': result[0],
            '2agre': result[1], 
            '3cons': result[2], 
            '4neur': result[3],
            '5open': result[4]
        })

In [ ]:
# hyperparameter sweep for SVM

sweep_config = {
    'method': 'grid'
}

metric = {
    'name': 'acc',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

parameters_dict = {
    'C': {
        'values': [1, 10, 100, 1000]
    },
    'kernel': {
        'values': ['linear', 'rbf', 'poly', 'sigmoid']
    },
    'gamma': {
        'values': ['scale', 0.001, 0.0001]
    }
}

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project='svm-sweep')

files = utils.get_files()
traits = utils.get_traits()

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        accuracy = []
        for i in range(5):
            result = []
            for testfile in files:
                x_train, y_train, x_test, y_test = load_data(testfile, i)

                clf = svm.SVC(C=config.C, gamma=config.gamma, kernel=config.kernel) 
                clf.fit(x_train, y_train) 

                result.append(1 if clf.score(x_test, y_test) >= 0.5 else 0)

            accuracy.append(sum(result) / len(result))
        acc = sum(accuracy) / len(accuracy)
        wandb.log({
            "acc": acc,
            "1extr": accuracy[0], 
            "2agre": accuracy[1],
            "3cons": accuracy[2], 
            "4neur": accuracy[3], 
            "5open": accuracy[4]
            })

wandb.agent(sweep_id, train)

In [4]:
# hyperparameter sweep for MLP

sweep_config = {
    'method': 'grid'
}

metric = {
    'name': 'acc',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_layer_sizes': {
        'values': [(256, 256), (256, 64), (512,)]
    },
    'activation': {
        'values': ['tanh', 'relu']
    },
    'solver': {
        'values': ['sgd', 'adam']
    },
    'alpha': {
        'values': [0.0001, 0.05]
    },
    'learning_rate': {
        'values': ['constant', 'adaptive']
    }
}

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project='mlp-sweep')

files = utils.get_files()
traits = utils.get_traits()

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        accuracy = []
        for i in range(5):
            result = []
            for testfile in files:
                x_train, y_train, x_test, y_test = load_data(testfile, i)

                clf = MLPClassifier(hidden_layer_sizes=config.hidden_layer_sizes, activation=config.activation, \
                    solver=config.solver, alpha=config.alpha, learning_rate=config.learning_rate, early_stopping=True) 
                clf.fit(x_train, y_train) 

                result.append(1 if clf.score(x_test, y_test) >= 0.5 else 0)

            accuracy.append(sum(result) / len(result))
        acc = sum(accuracy) / len(accuracy)
        wandb.log({
            "acc": acc,
            "1extr": accuracy[0], 
            "2agre": accuracy[1],
            "3cons": accuracy[2], 
            "4neur": accuracy[3], 
            "5open": accuracy[4]
            })

wandb.agent(sweep_id, train)

Create sweep with ID: r73z8z2k
Sweep URL: https://wandb.ai/r-yanagimoto/mlp-sweep/sweeps/r73z8z2k


wandb: Agent Starting Run: vo7zibah with config:
wandb: 	activation: tanh
wandb: 	alpha: 0.0001
wandb: 	hidden_layer_sizes: [256, 256]
wandb: 	learning_rate: constant
wandb: 	solver: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0.57692
2agre,0.46154
3cons,0.46154
4neur,0.5
5open,0.57692


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: my8p1ewz with config:
wandb: 	activation: tanh
wandb: 	alpha: 0.0001
wandb: 	hidden_layer_sizes: [256, 256]
wandb: 	learning_rate: constant
wandb: 	solver: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0.53846
2agre,0.53846
3cons,0.34615
4neur,0.42308
5open,0.38462


...

wandb: Agent Starting Run: t0fzx9d2 with config:
wandb: 	activation: relu
wandb: 	alpha: 0.05
wandb: 	hidden_layer_sizes: [512]
wandb: 	learning_rate: adaptive
wandb: 	solver: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0.57692
2agre,0.53846
3cons,0.53846
4neur,0.53846
5open,0.53846


wandb: Agent Starting Run: rx1l3thc with config:
wandb: 	activation: relu
wandb: 	alpha: 0.05
wandb: 	hidden_layer_sizes: [512]
wandb: 	learning_rate: adaptive
wandb: 	solver: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1extr,▁
2agre,▁
3cons,▁
4neur,▁
5open,▁
acc,▁
1extr,0.57692
2agre,0.5
3cons,0.34615
4neur,0.46154
5open,0.46154


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
